The next cell is to address the following error that arose after manually editing the central results file:
"json.decoder.JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 1789 (char 1788)"
This is for manual editing.

In [ ]:
import json
results_dict = {"Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__125, DeterministicImageNet": {"accuracy": 0.7619, "val_loss": 802.1857429970987}, "Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__125, ImageNetV2": {"accuracy": 0.6526, "val_loss": 258.0610283613205}, "./models/wiseft/ViTB32_8/zeroshot.pt, DeterministicImageNet": {"accuracy": 0.62934, "val_loss": 1290.983900883235}, "./models/wiseft/ViTB32_8/zeroshot.pt, ImageNetV2": {"accuracy": 0.551, "val_loss": 343.7056163549423}, "./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, DeterministicImageNet": {"accuracy": 0.75792, "val_loss": 928.6462971735746}, "./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, ImageNetV2": {"accuracy": 0.6412, "val_loss": 316.9530791044235}, "WSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, DeterministicImageNet": {"accuracy": 0.7568, "val_loss": 871.6008926071227}, "WSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, ImageNetV2": {"accuracy": 0.6535, "val_loss": 275.73413586616516}, "OSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, DeterministicImageNet": {"accuracy": 0.76194, "val_loss": -1.0}, "OSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, ImageNetV2": {"accuracy": 0.6527, "val_loss": -1.0}, "OAE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, DeterministicImageNet": {"accuracy": 0.82638, "val_loss": -1.0}, "OAE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt, ImageNetV2": {"accuracy": 0.7311, "val_loss": -1.0}}

# with open("central_results.txt","w") as file: # Commented out to prevent accidentally overwriting later results
#     file.write(json.dumps(results_dict)) 

The next cell is to create a csv file with accuracies for all the models that are based on the same two base models, including all the different versions of stacks built off of them.

In [6]:
import json
import time
import copy
import pandas as pd
import os

def saveManyComparisons(model1="./models/wiseft/ViTB32_8/zeroshot.pt",
                        model2 = "./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt",
                        eval_datasets = ["DeterministicImageNet","ImageNetV2"],
                        runNum1=125, runNum2=170):
    
    model_ckpts = [model1,model2]
    save = f"models/wiseft/stack_ViTB32_8_DeterministicImageNet_collected_results_{runNum1}_{runNum2}"
    # Create accuracy table like in stack.py, iterate through keys

    t0 = time.time()
    model_names = copy.deepcopy(model_ckpts)
    model_names.append("WSE__"+"__".join(model_ckpts)) # Weight space ensemble
    model_names.append("OSE__"+"__".join(model_ckpts)) # Output space ensemble
    model_names.append("OAE__"+"__".join(model_ckpts)) # Optimal alpha ensemble
    abbreviated_model_names = [f"model{i+1}" for i in range(len(model_ckpts))]+["WSE","OSE","OAE"]
    for runNum in range(runNum1,runNum2+1):
        model_names.append("Stack__"+"__".join(model_ckpts)+f"__{runNum}") # Stacking ensemble,"Stack"]
        abbreviated_model_names.append(f"Stack_{runNum}")

    accuracy_table = [[0 for j in model_names] for i in eval_datasets]
    val_loss_table = [[0 for j in model_names] for i in eval_datasets]
    for j in range(len(model_names)):
        for i in range(len(eval_datasets)):
            dataset_name = eval_datasets[i]
            accuracy_table[i][j],val_loss_table[i][j] = __get_accuracy_and_val_loss(model_names[j],dataset_name)

            # Save after each addition, so even if something goes wrong, what is completed so far will be saved.
            accuracy_df = pd.DataFrame(accuracy_table, eval_datasets, abbreviated_model_names)
            val_loss_df = pd.DataFrame(val_loss_table, eval_datasets, abbreviated_model_names)
            if save is not None:
                os.makedirs(save, exist_ok=True)
                accuracy_df.to_csv(os.path.join(save, 'accuracy_table.csv')) # Overwrites existing file
                val_loss_df.to_csv(os.path.join(save, 'val_loss_table.csv')) # Overwrites existing file
                print(f"Saved. i = {i}, j = {j}")

    # except KeyError as e:
    #     print("Some alphas not available.")
    #     print(e)

    tf = time.time()
    total_time = tf-t0
    time_file = os.path.join(save, 'time.txt')
    with open(time_file,'w') as file:
        file.write(f"Total time for saveComparisons function: {round(total_time,3)} seconds = {round(total_time/60,4)} minutes.")

In [3]:
def __get_accuracy_and_val_loss(model_name,eval_dataset_name):
    print(f"\nGetting accuracy and val loss of {model_name} on {eval_dataset_name}.\n")
    centralized_results_file = "./central_results.txt"
    key = f"{model_name}, {eval_dataset_name}"
    with open(centralized_results_file,"r") as file:
        results = json.loads(file.read())
        if key in results.keys():
            accuracy = results[key]["accuracy"]
            val_loss = results[key]["val_loss"]
        else:
            print(f"Key {key} not found in centralized_results_file.")
    return accuracy,val_loss

In [7]:
save = f"models/wiseft/stack_ViTB32_8_DeterministicImageNet_collected_results_125_170"
print(os.path.join(save, 'accuracy_table.csv'))

models/wiseft/stack_ViTB32_8_DeterministicImageNet_collected_results_125_170/accuracy_table.csv


In [8]:
saveManyComparisons()


Getting accuracy and val loss of ./models/wiseft/ViTB32_8/zeroshot.pt on DeterministicImageNet.

Saved. i = 0, j = 0

Getting accuracy and val loss of ./models/wiseft/ViTB32_8/zeroshot.pt on ImageNetV2.

Saved. i = 1, j = 0

Getting accuracy and val loss of ./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt on DeterministicImageNet.

Saved. i = 0, j = 1

Getting accuracy and val loss of ./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt on ImageNetV2.

Saved. i = 1, j = 1

Getting accuracy and val loss of WSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt on DeterministicImageNet.

Saved. i = 0, j = 2

Getting accuracy and val loss of WSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt on ImageNetV2.

Saved. i = 1, j = 2

Getting accuracy and val loss of OSE__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt on DeterministicImageNet.

Saved. i = 0, j = 3

Ge

Saved. i = 0, j = 24

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__144 on ImageNetV2.

Saved. i = 1, j = 24

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__145 on DeterministicImageNet.

Saved. i = 0, j = 25

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__145 on ImageNetV2.

Saved. i = 1, j = 25

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__146 on DeterministicImageNet.

Saved. i = 0, j = 26

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wiseft/ViTB32_8/finetuned/checkpoint_10.pt__146 on ImageNetV2.

Saved. i = 1, j = 26

Getting accuracy and val loss of Stack__./models/wiseft/ViTB32_8/zeroshot.pt__./models/wise